# Lineage tracing identifies heterogeneous hepatoblast contribution to cell lineages and postembryonic organ growth dynamics

In [this paper](https://www.biorxiv.org/content/10.1101/2023.01.18.524321v1), a simple model is used to analyze the ratio of cell types during liver development in Zebrafish. The aim here is to generate a similar model, in a simpler way. I will go for some deterministic modeling instead of stochastic. The results should be the same for the mean behaviour but we won't have the variance. We could then try some agent-based modeling if we wanted to get the variance or add some other functionalities... or just for the sake of it.

Remember that to run the cells you have to select it and press ctrl + enter.

In [ ]:
!conda install -c alubbock bionetgen -y

In [46]:
import numpy as np
import matplotlib.pyplot as plt

from pysb.simulator import ScipyOdeSimulator
import simple_model as ober_model

from ipywidgets import interact, Layout, IntSlider

Simulator = ScipyOdeSimulator(ober_model.model)

The following cell will run the model, and you can select the values for each aprametrs as you see fit. They are not exactly the same parameters, but they are closely related.

### Cell division rates

1. hepatoblast_division_rate is the mean time interval in hours for an hepatoblast to divide leading to two hepatoblasts. Default is set to 0 as they are supposed to differentiate after dividing.
2. hepatocyte_division_rate is the mean time interval in hours for an hepatocyte to divide leading to two hepatocytes. Default is set to 24 hours.
3. cholangiocyte_division_rate is the mean time interval in hours for a cholangiocyte to divide leading to two cholangiocytes. Default is set to 24 hours.

### Hepatoblast differentiation rates

1. hepatoblast_bipotent_division_rate is the mean time interval in hours for an hepatoblast to divide and differentiate into an hepatocyte and a cholangiocyte. Default is set to 24 hours.
2. hepatoblast_unipotent_hepatocyte_division_rate is the mean time interval in hours for an hepatoblast to divide and differentiate into two hepatocytes. Default is set to 6 hours.
3. hepatoblast_unipotent_cholangiocyte_division_rate is the mean time interval in hours for an hepatoblast to divide and differentiate into two cholangiocytes. Default is set to 0 hours.

In [63]:
def plot(hepatoblast_division_rate=0, hepatocyte_division_rate=24, cholangiocyte_division_rate=24, 
         hepatoblast_bipotent_division_rate=24, hepatoblast_unipotent_hepatocyte_division_rate=0, hepatoblast_unipotent_cholangiocyte_division_rate=0):
    
    for name, rate in [('hepatoblast_division_rate', hepatoblast_division_rate), 
                       ('hepatocyte_division_rate', hepatocyte_division_rate), 
                       ('cholangiocyte_division_rate', cholangiocyte_division_rate), 
                       ('hepatoblast_bipotent_division_rate', hepatoblast_bipotent_division_rate), 
                       ('hepatoblast_unipotent_hepatocyte_division_rate', hepatoblast_unipotent_hepatocyte_division_rate), 
                       ('hepatoblast_unipotent_cholangiocyte_division_rate', hepatoblast_unipotent_cholangiocyte_division_rate)]:
        rate = np.log(2) / rate if rate != 0 else 0
        Simulator.model.parameters[name].value = rate
            
    time = np.linspace(0, 144, 145)
    sim_result = Simulator.run(tspan=time)
    
    fig, axs = plt.subplots(2, 1,figsize=(8, 12), sharex=True)
    
    sim_result.dataframe.plot(logy=True, ylim=(1, sim_result.dataframe.max().max()), ax=axs[0])
    sim_result.dataframe.div(sim_result.dataframe.sum(axis=1) / 2, axis=0).plot(ax=axs[1])
    
    plt.subplots_adjust(hspace=0)
    plt.xlabel('Time (hours)')
    axs[0].set_ylabel('Number of Cells')
    axs[1].set_ylabel('Percentage of Cells')
    plt.show()
    
    print('ratio = 1:%0.1f' % (sim_result.dataframe['Hepatocytes'].values[-1] / sim_result.dataframe['Cholangiocytes'].values[-1]))

interact(plot, hepatoblast_division_rate=IntSlider(0, 0, 72, 6, style={'description_width': 'auto'}, layout=Layout(width='700px')),
          hepatocyte_division_rate=IntSlider(24, 0, 72, 6, style={'description_width': 'auto'}, layout=Layout(width='700px')), 
          cholangiocyte_division_rate=IntSlider(24, 0, 72, 6, style={'description_width': 'auto'}, layout=Layout(width='700px')), 
          hepatoblast_bipotent_division_rate=IntSlider(24, 0, 72, 6, style={'description_width': 'auto'}, layout=Layout(width='700px')), 
          hepatoblast_unipotent_hepatocyte_division_rate=IntSlider(6, 0, 72, 6, style={'description_width': 'auto'}, layout=Layout(width='700px')), 
          hepatoblast_unipotent_cholangiocyte_division_rate=IntSlider(0, 0, 72, 6, style={'description_width': 'auto'}, layout=Layout(width='700px')));

interactive(children=(IntSlider(value=0, description='hepatoblast_division_rate', layout=Layout(width='700px')…

## TO DO

1. We could use the hepatoblast division rate to determine how fast they divide and then set percentages for the fates as in the paper. Ideally we would have to automatically update the parameters so that they are never above 100% in total.

2. We should use functions for the rates and not constant values as the second figure already shows a time dependence.